![alt text](./img/header.png)

# Saltwater Intrusion Exercise

A growing coastal city is interested in developing a wellfield in order to provide potable drinking water to its residents.  You have been contracted by the city to perform a numerical analysis to determine the effects of groundwater withdrawals on the position of the saltwater interface.  The city is particularly interested in knowing if the planned withdrawal rate will result in pumped water that exceeds the drinking water standard for chloride, which is 250 mg/L.  In addition, you have been asked by the city to delineate a wellfield protection zone by determining the approximate recharge area for the wellfield.

![alt text](./img/exB_fig1.png)

The study area consists of a 10 by 10 km area of a coastal plain aquifer system centered on the proposed wellfield (Figure 1).  The hydrogeology of the area consists of a shallow and deep aquifer, and both aquifers intersect the sea floor (Figure 2).  The two aquifers are separated by a discontinuous confining layer that thickens to the east.  The confining layer is present in the eastern part of the study area, but is absent in the western part.  A low permeability unit, which slopes downward to the east, underlies the deep aquifer and can be considered a no flow boundary.  Extensive field investigations have been performed, and the aquifer properties for the different units are summarized in the table below.  Annual average recharge was determined to be 25.4 cm/yr (10 in/yr).

![alt text](./img/exB_fig2.png)

Unfortunately, due to limitations in available property, the city has only one option for wellfield location, which is shown on the map in Figure 1.  The city is hoping to capture at least 25% of the recharge for the area, but there is an obvious concern that excessive pumping from the deep aquifer could cause the saltwater interface to move inland and contaminate the wellfield.

Note that we will now use the following calibrated values.  The original values that we used are shown in parentheses.

  Unit            | $K_h$ |  $K_v$      |  $S$        | $S_y$ | $n$   | $\alpha_l$ | $\alpha_v$ 
  --------------- | ----- | ----------- | ----------- | ----- | ----- | ---------- | ---------- 
  Shallow aquifer | 50 (100)   | 0.5 (1)           | $1x10^{-5}$ | 0.2   | 0.2   | 10         | 1          
  Confining unit  | 0.001 | 0.001       | $1x10^{-5}$ | 0.2   | 0.2   | 10         | 1          
  Deep aquifer    | 1000 (2000)  | 100 (200)         | $1x10^{-5}$ | 0.2   | 0.2   | 10         | 1          



In [ ]:
# Setup the python environment
%matplotlib inline
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy
import config

## Exercise C -- Design and run a 3D model to determine the effects of groundwater pumping on saltwater interface movement

In this part of the exercise, you will determine if the recommended pumping rate can be sustained for 20 years without exceeding the chloride drinking water standard of 250 mg/L.  The general procedure for this exercise will be to convert the calibrated model from exSEAWAT_B into a 3D model, and then perform simulations with and without pumping.


### Part C1 -- Convert the calibrated model into a 3D model of the study area

The first step here is to set the initial heads and concentrations for the 3D model using the results from the previous 2D calibrated simulation.  Then, recreate the model as a 3D model. Lastly, perform a simulation without pumping to verify that the initial concentrations and heads are at equilibrium for the 3D situation.  The last step is optional, however, it is good practice to ensure that the conversion from 2D to 3D occurred without error.

In [ ]:
# Read the heads and concentrations from the last simulation
pth = os.path.join('data', 'exSEAWAT_C')
fname = os.path.join(pth, 'b4.hds')
hdobj = flopy.utils.binaryfile.HeadFile(fname)
times = hdobj.get_times()
head = hdobj.get_data(totim=times[-1])
hdobj.close()

fname = os.path.join(pth, 'MT3D001.UCN')
cnobj = flopy.utils.binaryfile.UcnFile(fname)
times = cnobj.get_times()
conc = cnobj.get_data(totim=times[-1])
cnobj.close()

print('Head array shape is {}.'.format(head.shape))
print('Concentration array shape is {}.'.format(conc.shape))

In [ ]:
# Data path
datapth = os.path.join('data', 'exSEAWAT_C')

# Grid information
nlay = 23
nrow = 100
ncol = 100
delr = 100.
delc = 100.
top = 2.5
botm = np.repeat(np.linspace(-2.5, -112.5, nlay), nrow * ncol).reshape((nlay, nrow, ncol))

# Temporal discretization
nper = 1
nstp = 1
perlen = 7300.  # 20 years

# Ibound
fname = os.path.join(datapth, 'ibound.txt')
ibound2d = np.loadtxt(fname).reshape((nlay, 1, ncol))
ibound = np.repeat(ibound2d, nrow, axis=1)

# Starting heads
strt = np.repeat(head, nrow, axis=1)

# Hydraulic properties
fname = os.path.join(datapth, 'hk.txt')
hk2d = np.loadtxt(fname).reshape((nlay, 1, ncol))
hk = np.repeat(hk2d, nrow, axis=1)

fname = os.path.join(datapth, 'vk.txt')
vk2d = np.loadtxt(fname).reshape((nlay, 1, ncol))
vk = np.repeat(vk2d, nrow, axis=1)

ss = 1.e-5
sy = 0.2

# Recharge
rech = 25.4 / 100. / 365.  # convert cm/yr to m/day

# Transport
dt0 = 25.
sconc = np.repeat(conc, nrow, axis=1)

icbund = np.abs(ibound)
ssm_data = {}
itype = flopy.mt3d.Mt3dSsm.itype_dict()
layers, rows, columns = np.where(ibound < 0)
ssm_per1 = []
for k, i, j in zip(layers, rows, columns):
    ssm_per1.append((k, i, j, 35., itype['CHD']))
ssm_data[0] = ssm_per1

In [ ]:
# Use the values required to calibrate the model
hk_shallow = 100 * 0.5
hk_conf = 0.001
hk_deep = 2000 * 0.5
vk_shallow = 1.0 * 0.5
vk_conf = 0.001
vk_deep = 200 * 0.5

# This will setup the hk2 and vk2 and make the substitutions
kzone = np.ones((nlay, nrow, ncol), dtype=np.int)
idx_shallow = np.where(np.float32(hk) == 100)
idx_conf = np.where(np.float32(hk) == 0.001)
idx_deep = np.where(np.float32(hk) == 2000)
hk2 = np.ones((nlay, nrow, ncol), dtype=np.float32) * -999
vk2 = np.ones((nlay, nrow, ncol), dtype=np.float32) * -999
hk2[idx_shallow] = hk_shallow
hk2[idx_conf] = hk_conf
hk2[idx_deep] = hk_deep
vk2[idx_shallow] = vk_shallow
vk2[idx_conf] = vk_conf
vk2[idx_deep] = vk_deep

In [ ]:
# Build the flopy SEAWAT model
model_ws = os.path.join('work', 'exSEAWAT_C')
if not os.path.isdir(model_ws):
    os.mkdir(model_ws)

# make list of observation locations
obs = np.array([[19, 49, 49]])
modelname = 'c1'
m = flopy.seawat.Seawat(modelname, model_ws=model_ws, exe_name=config.swexe)
dis = flopy.modflow.ModflowDis(m, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, 
                               top=top, botm=botm, perlen=perlen, nstp=nstp)
bas = flopy.modflow.ModflowBas(m, ibound, strt=strt)
lpf = flopy.modflow.ModflowLpf(m, laytyp=1, hk=hk2, vka=vk2, ss=ss, sy=sy, ipakcb=53)
rch = flopy.modflow.ModflowRch(m, rech=rech, ipakcb=53)
oc = flopy.modflow.ModflowOc(m, stress_period_data={(0, 0): ['save head', 'save budget']})
pcg = flopy.modflow.ModflowPcg(m, hclose=1.e-3, rclose=1e4)
vdf = flopy.seawat.SeawatVdf(m, mtdnconc=1, mfnadvfd=1, nswtcpl=0, iwtable=0, 
                             densemin=0., densemax=0., denseslp=25./35., denseref=1000.)
btn = flopy.mt3d.Mt3dBtn(m, nlay, nrow, ncol, nper, laycon=lpf.laytyp, htop=top, 
                        dz=dis.thickness.get_value(), prsity=0.2, icbund=icbund,
                        sconc=sconc, nprs=-10, dt0=25., obs=obs)
adv = flopy.mt3d.Mt3dAdv(m, mixelm=0, percel=0.5)
dsp = flopy.mt3d.Mt3dDsp(m, al=10., trpt=0.1, trpv=0.1, dmcoef=0.)
ssm = flopy.mt3d.Mt3dSsm(m, crch=0, stress_period_data=ssm_data)
gcg = flopy.mt3d.Mt3dGcg(m, isolve=2, cclose=1.e-6)

In [ ]:
# Make plot of the grid
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
linecollection = mm.plot_grid()
patchcollection = mm.plot_ibound()

In [ ]:
# Make color flood plot of hydraulic conductivity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
hkpatchcollection = mm.plot_array(np.log(m.lpf.hk.array), cmap='viridis')
linecollection = mm.plot_grid()
patchcollection = mm.plot_ibound()
cb = plt.colorbar(hkpatchcollection)

In [ ]:
# write the input files
m.write_input()

In [ ]:
# run the model
success, buff = m.run_model(silent=True)

In [ ]:
# Extract salinity
fname = os.path.join(model_ws, 'MT3D001.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])
conc[np.where(ibound != 1)] = np.nan

In [ ]:
# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
#linecollection = mm.plot_grid()
#patchcollection = mm.plot_ibound()
cb = plt.colorbar(cpatchcollection)

In [ ]:
# Load the mas file and make a plot of total mass in aquifer versus time
fname = os.path.join(model_ws, 'MT3D001.MAS')
mas = flopy.mt3d.Mt3dms.load_mas(fname)
f = plt.figure()
ax = f.add_subplot(1, 1, 1)
lines = ax.plot(mas.time, mas.total_mass)

### C1 Questions
 
1. How do the results of the 3D simulation compare to the cross section simulation?
2. Animate the results of this simulation using Model Viewer

### Part C2 -- Evaluate the effects of saltwater intrusion in response to groundwater withdrawal

The city has determined that they would like to capture about 25% of the groundwater recharge.  In this part of the exercise, you will perform a simulation to determine if the drinking water standard for chloride would be violated with this withdrawal rate.  Your simulation period will remain at 20 years, as the city’s water supply plan only focuses on the next 20 years.

In [ ]:
# Calculate the well pumping rate
prate = rech * 10000 * 7000 * 0.25
print('Well pumping rate is {}'.format(prate))

In [ ]:
# This code block will make a new model called c2 and then run it
m.name = 'c2'
well_spdata = {}
wellsp0 = [[19, 49, 49, -prate]]
well_spdata[0] = wellsp0
wel = flopy.modflow.ModflowWel(m, stress_period_data=well_spdata, ipakcb=53)
m.write_input()
m.run_model(silent=True)

Concentrations at the well are stored in the file MT3D001.OBS.  This file contains the concentration in cell (20, 50, 50) for each transport time step.  Note that this concentration is in total dissolved solids (TDS).  The following code block will import this file and prepare a graph of TDS versus time.  You will need to make the conversion to chloride concentration by assuming the chloride concentration in seawater is about 19,000 mg/L.  Note that the TDS of seawater is about 35 g/L (the concentration value assigned to the ocean boundary).

In [ ]:
# Load the obs file and make a plot of salinity at the well
fname = os.path.join(model_ws, 'MT3D001.OBS')
obsdata = flopy.mt3d.Mt3dms.load_obs(fname)
f = plt.figure()
ax = f.add_subplot(1, 1, 1)
lines = ax.plot(obsdata.time, obsdata['(20, 50, 50)'])

### C2 Questions
 
1. After 20 years of pumping, has the drinking water limit for chloride been violated?
2. What was the average rate of inland movement of the 250 mg/L chloride concentration at the base of the aquifer? 

### Part C3 -- Evaluate the effects of density on the prediction

It is difficult to determine *a priori* whether or not density variations are necessary to incorporate in a simulation of coastal groundwater flow.  The purpose of this next, optional analysis is to shut off the variable density component in SEAWAT and quantify the effects on the simulated concentration in cell (20, 50, 50) and on the water budget.  There are several ways to perform a simulation where the density effects are neglected: (1) make the DENSESLP equal to zero, (2) remove the vdf package from the simulation, or (3) use MODFLOW and MT3DMS.  You might try performing the first way as it will be easy to do within the notebook and the current setup.  Prior to performing the simulations, think about what you might expect to happen in a constant-density simulation of our saltwater intrusion problem.

### Part C4 -- Determine wellfield protection area

Numerical models, such as the SEAWAT model developed for this exercise, can be used to delineate the recharge areas for groundwater pumping wells.  In this exercise, you will use MODPATH to perform forward and backward particle tracking analysis to delineate the recharge area for the city’s proposed wellfield.  The cell-by-cell flow file that was created during the 20 year pumping simulation from SWI_IIB contains the intercell flows for the entire simulation.  Flows from the last transport time step will be used to determine the well-field protection area.  By performing this particle tracking analysis, we are assuming that these intercell flows remained constant throughout the simulation.  If we wanted to see the effects of transient intercell flows on the particle movement, we would need to rerun the SEAWAT simulation with additional MODFLOW time steps so that the intercell flows could be saved more frequently.

#### Pathline analysis

The first step will be to create a MODPATH model from the SEAWAT model and run backward pathlines from the pumping well. To create the MODPATH model a `flopy.modpath.Modpath` object instance is created. The a MODPATH basic package is created using the ibound array from the SEAWAT model and porosity for the aquifer layers. A constant porosity of 0.2 is used. Then a MODPATH simulation package is created using the well package. The `create_mpsim()` method creates the particles for the backward pathlines using the cells will wells by adding particles on all 6 cell faces.

In [ ]:
mpname = m.name + 'mp'
mp = flopy.modpath.Modpath(modelname=mpname, exe_name=config.mpexe, modflowmodel=m, model_ws=model_ws)
mp.dis_unit = 11
mpbas = flopy.modpath.ModpathBas(mp, hnoflo=m.bas6.hnoflo, hdry=m.lpf.hdry, 
                                 ibound=m.bas6.ibound.array, prsity=0.2)
sim = mp.create_mpsim(trackdir='backward', simtype='pathline', packages='WEL')

Once the MODPATH model object, basic package, and simulation package are created then the MODPATH input files can be written and the model can be run.

In [ ]:
mp.write_input()
mp.run_model()

Use `flopy.utils.PathlineFile` to read MODPATH pathline output.

In [ ]:
pthfile = os.path.join(model_ws, mp.sim.pathline_file)
pthobj = flopy.utils.PathlineFile(pthfile)
plines = pthobj.get_alldata()

Plot the results using the `ModelMap` `plot_pathline()` method. Use the optional `travel_time` argument to plot pathlines for times less than or equal to 10000 days.

In [ ]:
f = plt.figure(figsize=(10, 10))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelMap(model=m, ax=ax, layer=19)
qmw = mm.plot_bc('WEL', zorder=10)
mm.plot_grid()
lc = mm.plot_pathline(plines, layer='all', colors='blue', travel_time='<= 10000')

#### Endpoint analysis

An endpoint is essentially the same as a pathline analysis except in this case we will start the particles at the top of the model, where recharge is applied, and run in a forward direction from recharge points to discharge points.

In [ ]:
mpname = modelname + 'mp'
mp = flopy.modpath.Modpath(modelname=mpname, exe_name=config.mpexe, modflowmodel=m, model_ws=model_ws)
mp.dis_unit = 11
mpbas = flopy.modpath.ModpathBas(mp, hnoflo=m.bas6.hnoflo, hdry=m.lpf.hdry, 
                                 ibound=m.bas6.ibound.array, prsity=0.2)
sim = mp.create_mpsim(trackdir='forward', simtype='endpoint', packages='RCH')

Again once the MODPATH model object, basic package, and simulation package are created then the MODPATH input files can be written and the model can be run.

In [ ]:
mp.write_input()
mp.run_model()

Use `flopy.utils.EndpointFile` to read MODPATH pathline output.

In [ ]:
endfile = os.path.join(model_ws, mp.sim.endpoint_file)
endobj = flopy.utils.EndpointFile(endfile)
ept = endobj.get_alldata()

Plot the endpoint results using the `ModelMap` `plot_endpoint()` method. Recharge areas can be identified by selecting the starting location of particles that end in the well. This is accomplished by setting the `direction` argument to `'starting'`, `selection_direction` to `'ending'` and using a `selection` tuple at the well `(19, 49, 49)`.

In [ ]:
f = plt.figure(figsize=(10, 10))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelMap(model=m, ax=ax, layer=0)
qib = mm.plot_inactive()
mm.plot_endpoint(ept, direction='starting', selection_direction='ending', selection=(19, 49, 49), size=1)

### C4 Questions
 
1. Evaluate visualizing different pathline travel_times.
2. Use ModelViewer to look at the pathlines.
3. Why does the area covered by the pathlines cover a larger area that the area identified by the endpoint analysis?
4. How does the porosity affect the analysis?